In [1]:
import re

with open('inputs/22.txt') as file:
    input = file.read()


In [2]:
%%time

def shuffle(deck, rules):
    for s in rules:
        if s == 'deal into new stack':
            deck.reverse()
            continue
        match = re.match(r'cut (-?\d+)', s)
        if match != None:
            n = int(match.group(1))
            deck = deck[n:]+deck[:n]
            continue
        match = re.match(r'deal with increment (\d+)', s)
        if match != None:
            n = int(match.group(1))
            ndeck = [0]*len(deck)
            for i in range(len(deck)):
                ndeck[(i*n)%len(deck)] = deck[i]
            deck = ndeck
    return deck

deck = shuffle(list(range(10007)), input.split('\n'))
deck.index(2019)


Wall time: 103 ms


2480

In [3]:
%%time

# Uh-oh...
119315717514047 * 101741582076661


Wall time: 0 ns


12139369866491111222698357067